In [1]:
from docplex.mp.model import Model as Model_cpx
import gurobipy as gp
from gurobipy import Model as Model_grb, GRB, quicksum
import math
import time

### Parser que lee el archivo de instancia y retorna un diccionario con los datos.

In [2]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]

    num_customers, num_depots = int(lines[0]), int(lines[1])
    depots = [tuple(map(float, lines[i].split())) for i in range(2, 2 + num_depots)]
    customers = [tuple(map(float, lines[i].split())) for i in range(2 + num_depots, 2 + num_depots + num_customers)]
    vehicle_capacity = float(lines[2 + num_depots + num_customers])
    depot_capacities = list(map(float, lines[3 + num_depots + num_customers:3 + 2 * num_depots + num_customers]))
    customer_demands = list(
        map(float, lines[3 + 2 * num_depots + num_customers:3 + 2 * num_depots + 2 * num_customers]))
    depot_opening_costs = list(
        map(float, lines[3 + 2 * num_depots + 2 * num_customers:3 + 3 * num_depots + 2 * num_customers]))
    route_opening_cost = float(lines[3 + 3 * num_depots + 2 * num_customers])

    all_points = depots + customers
    distance_matrix = [
        [math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) for x2, y2 in all_points] for x1, y1 in all_points
    ]

    return {
        "num_customers": num_customers,
        "num_depots": num_depots,
        "depots": depots,
        "customers": customers,
        "vehicle_capacity": vehicle_capacity,
        "depot_capacities": depot_capacities,
        "customer_demands": customer_demands,
        "depot_opening_costs": depot_opening_costs,
        "route_opening_cost": route_opening_cost,
        "distance_matrix": distance_matrix,
    }

# Modelo SCF (Single Commodity Flow).

In [3]:
def modelo_scf_cplex(parsed_data):
    """
    Implementación del modelo SCF (Single Commodity Flow) usando CPLEX.

    Parámetros:
        parsed_data (dict): Diccionario con los datos parseados.

    Retorna:
        Model: Modelo de CPLEX con el problema SCF formulado.
    """
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    opening_cost_route = parsed_data["route_opening_cost"]
    costs = parsed_data["distance_matrix"]

    # Crear el modelo
    mdl = Model_cpx(name="SCF_cpx")

    # Nodos totales (depósitos + clientes)
    nodos = list(range(m + n))

    # Variables
    x = mdl.binary_var_matrix(nodos, nodos, name="x")  # Ruta entre nodos
    f = mdl.continuous_var_matrix(nodos, nodos, name="f")  # Flujo de producto
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo: minimizar costos de transporte, apertura y uso de rutas
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in nodos for j in nodos if i != j) +  # Costos de transporte
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m)) +  # Costos de apertura de depósitos
        opening_cost_route * mdl.sum(y[d] for d in range(m))  # Costos de rutas
    )

    # Restricciones
    # 1. Cada cliente es atendido exactamente una vez
    for i in range(m, m + n):
        mdl.add_constraint(mdl.sum(x[i, j] for j in nodos if j != i) == 1)  # Salen
        mdl.add_constraint(mdl.sum(x[j, i] for j in nodos if j != i) == 1)  # Entran

    # 2. Las rutas deben comenzar y terminar en depósitos abiertos
    for d in range(m):
        mdl.add_constraint(mdl.sum(x[d, j] for j in range(m, m + n)) <= y[d])
        mdl.add_constraint(mdl.sum(x[j, d] for j in range(m, m + n)) <= y[d])

    # 3. Restricciones de flujo para garantizar balance
    for i in range(m, m + n):  # Para cada cliente
        mdl.add_constraint(
            mdl.sum(f[i, j] for j in nodos if j != i) -
            mdl.sum(f[j, i] for j in nodos if j != i) == customer_demands[i - m]
        )
    for i, j in [(i, j) for i in nodos for j in nodos if i != j]:
        mdl.add_constraint(f[i, j] <= vehicle_capacity * x[i, j])  # Flujo limitado por capacidad del vehículo

    # 4. Capacidad máxima de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(f[d, j] for j in range(m, m + n)) <= depot_capacities[d]
        )

    return mdl


In [4]:
def modelo_scf_gurobi(parsed_data):
    """
    Implementación del modelo SCF (Single Commodity Flow) usando Gurobi.

    Parámetros:
        parsed_data (dict): Diccionario con los datos parseados.

    Retorna:
        Model: Modelo de Gurobi con el problema SCF formulado.
    """
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    opening_cost_route = parsed_data["route_opening_cost"]
    costs = parsed_data["distance_matrix"]

    # Crear el modelo
    mdl = Model_grb("SCF")

    # Nodos totales (depósitos + clientes)
    nodos = list(range(m + n))

    # Variables
    x = mdl.addVars(nodos, nodos, vtype=GRB.BINARY, name="x")  # Ruta entre nodos
    f = mdl.addVars(nodos, nodos, vtype=GRB.CONTINUOUS, name="f")  # Flujo de producto
    y = mdl.addVars(m, vtype=GRB.BINARY, name="y")  # Uso de depósitos

    # Función objetivo: minimizar costos de transporte, apertura y uso de rutas
    mdl.setObjective(
        quicksum(costs[i][j] * x[i, j] for i in nodos for j in nodos if i != j) +  # Costos de transporte
        quicksum(opening_costs_depots[d] * y[d] for d in range(m)) +  # Costos de apertura de depósitos
        opening_cost_route * quicksum(y[d] for d in range(m)),  # Costos de rutas
        GRB.MINIMIZE
    )

    # Restricciones
    # 1. Cada cliente es atendido exactamente una vez
    for i in range(m, m + n):
        mdl.addConstr(quicksum(x[i, j] for j in nodos if j != i) == 1)  # Salen
        mdl.addConstr(quicksum(x[j, i] for j in nodos if j != i) == 1)  # Entran

    # 2. Las rutas deben comenzar y terminar en depósitos abiertos
    for d in range(m):
        mdl.addConstr(quicksum(x[d, j] for j in range(m, m + n)) <= y[d])
        mdl.addConstr(quicksum(x[j, d] for j in range(m, m + n)) <= y[d])

    # 3. Restricciones de flujo para garantizar balance
    for i in range(m, m + n):  # Para cada cliente
        mdl.addConstr(
            quicksum(f[i, j] for j in nodos if j != i) -
            quicksum(f[j, i] for j in nodos if j != i) == customer_demands[i - m]
        )
    for i, j in [(i, j) for i in nodos for j in nodos if i != j]:
        mdl.addConstr(f[i, j] <= vehicle_capacity * x[i, j])  # Flujo limitado por capacidad del vehículo

    # 4. Capacidad máxima de los depósitos
    for d in range(m):
        mdl.addConstr(
            quicksum(f[d, j] for j in range(m, m + n)) <= depot_capacities[d]
        )

    return mdl

# Modelo CDA (Capacitated Depot Allocation).

In [5]:
def modelo_cda_cplex(parsed_data):
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    costs = parsed_data["distance_matrix"]

    mdl = Model_cpx(name="CDA_cpx")

    # Variables
    v = mdl.binary_var_matrix(n, m, name="v")  # Cliente asignado a depósito
    x = mdl.binary_var_matrix(n + m, n + m, name="x")  # Uso de arcos
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in range(n + m) for j in range(n + m)) +
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m))
    )

    # Restricciones
    # 1. Cada cliente asignado a un único depósito
    for i in range(n):
        mdl.add_constraint(mdl.sum(v[i, d] for d in range(m)) == 1)

    # 2. Si un cliente está asignado a un depósito, el depósito debe estar activo
    for i in range(n):
        for d in range(m):
            mdl.add_constraint(v[i, d] <= y[d])

    # 3. Relación entre asignaciones y rutas
    for i in range(n):
        for j in range(n):
            for d in range(m):
                mdl.add_constraint(v[i, d] + x[i + m, j + m] <= v[j, d] + 1)

    # 4. Capacidades de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(customer_demands[i] * v[i, d] for i in range(n)) <= depot_capacities[d]
        )

    # 5. Capacidad del vehículo
    for i, j in [(i, j) for i in range(n + m) for j in range(n + m) if i != j]:
        mdl.add_constraint(x[i, j] <= vehicle_capacity)

    return mdl


In [6]:
def modelo_cda_gurobi(parsed_data):
    # Extract parameters from parsed data
    n = parsed_data["num_customers"]  # Number of customers
    m = parsed_data["num_depots"]  # Number of depots
    depot_coords = parsed_data["depots"]  # List of depot coords (e.g., [(0, 0), (1, 1), ...])
    customer_coords = parsed_data["customers"]  # List of customer coords (e.g., [(0, 0), (1, 1), ...])
    vehicle_capacity = parsed_data["vehicle_capacity"]  # Capacity of a vehicle
    depot_capacities = parsed_data["depot_capacities"]  # List of depot capacities
    customer_demands = parsed_data["customer_demands"]  # List of customer demands
    opening_costs_depots = parsed_data["depot_opening_costs"]  # List of depot opening costs
    route_opening_cost = parsed_data["route_opening_cost"]  # Cost of opening a route
    costs = parsed_data["distance_matrix"]  # Distance matrix

    # listas de índices para depots y customers
    customers = range(n)
    depots = range(n, n + m)

    # Hacer que keys() funcione
    arc_indices = [(i, j) for i in range(m + n) for j in range(m + n) if i != j]

    # Create a Gurobi model
    model = Model_grb("CDA_grb")

    # Variables
    x = model.addVars(arc_indices, vtype=GRB.BINARY,
                      name="x")  # Variable Binaria que indica si el arco de i a j está siendo usado
    v = model.addVars([(i, d) for i in customers for d in depots], vtype=GRB.CONTINUOUS,
                      name="v")  # Variable continua que indica si el cliente está asignado a un depot d
    y = model.addVars(depots, vtype=GRB.BINARY, name="y")  # Variable binaria que indica si el depot d está abierto

    # Funcion objetivo

    model.setObjective(
        gp.quicksum(costs[i][j] * x[i, j] for i, j in arc_indices) +  # Minimizar el costo de la ruta +
        gp.quicksum(opening_costs_depots[d - n] * y[d] for d in depots) +  # El costo del inicio del depot +
        route_opening_cost * gp.quicksum(x[d, i] for d in depots for i in customers),  # El inicio de la ruta.
        GRB.MINIMIZE
    )

    # Restricción CDA
    # Asegura que cada cliente este asignado a un sólo depot
    model.addConstrs((gp.quicksum(v[i, d] for d in depots) == 1 for i in customers), "client_assignment")

    # Asegura que si se ocupa un arco [i,d], el cliente [d,i] o [i,d] estará ahí
    model.addConstrs((x[d, i] <= v[i, d] for d in depots for i in customers), "client_inclusion_outgoing")
    model.addConstrs((x[i, d] <= v[i, d] for d in depots for i in customers), "client_inclusion_incoming")

    # Restriccion que elimina subtours
    model.addConstrs((v[i, d] + x[i, j] <= v[j, d] + 1 for d in depots for i in customers for j in customers if i != j),
                     "path_elimination")

    # Restricciones genéricas
    # Restriccion capacidad depot
    model.addConstrs(
        (gp.quicksum(customer_demands[i] * v[i, d] for i in customers) <= depot_capacities[d - n] * y[d] for d in
         depots),
        "depot_capacity")

    # Restriccion capacidad vehiculo
    model.addConstrs(
        (gp.quicksum(customer_demands[i] * x[i, j] for i in customers for j in customers if i != j) <= vehicle_capacity
         for
         d in depots), "vehicle_capacity")

    # Restriccion depot abierto
    model.addConstrs((v[i, d] <= y[d] for d in depots for i in customers), "depot_open")

# Ejecución de los modelos.

In [7]:
# archivo de resultados
file = open('results.csv', 'w')
file.write(
    "Modelo,Benchmark,Instancia,Número de Variables,Número de Restricciones,Valor Función Objetivo,Tiempo de Cómputo (s)\n")
file.flush()

In [8]:
# DATA
instances = {'B1': ['coord20-5-1.dat', 'coord100-5-3b.dat', 'coord200-10-3b.dat'],
             'B2': ['coordP111112.dat', 'coordP123222.dat', 'coordP133222.dat'],
             'B3': ['coordChrist50.dat', 'coordDas150.dat', 'coordMin134.dat']}

parsed_data = {'B1': [parse_file('Instances/Benchmark_1/' + inst) for inst in instances['B1']],
               'B2': [parse_file('Instances/Benchmark_2/' + inst) for inst in instances['B2']],
               'B3': [parse_file('Instances/Benchmark_3/' + inst) for inst in instances['B3']]}

In [ ]:
# Creación de modelos
# modelos = {'B1': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B1']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B1']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B1']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B1']]}},
#            'B2': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B2']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B2']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B2']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B2']]}},
#            'B3': {'cplex': {'SCF': [modelo_scf_cplex(data) for data in parsed_data['B3']],
#                             'CDA': [modelo_cda_cplex(data) for data in parsed_data['B3']]},
#                   'gurobi': {'SCF': [modelo_cda_gurobi(data) for data in parsed_data['B3']],
#                              'CDA': [modelo_cda_gurobi(data) for data in parsed_data['B3']]}}}

#modelos = {'SCF': {'cplex': [modelo_scf_cplex(data) for B in parsed_data for data in parsed_data[B]],
#                   'gurobi': [modelo_scf_gurobi(data) for B in parsed_data for data in parsed_data[B]]},
#           'CDA': {'cplex': [modelo_cda_cplex(data) for B in parsed_data for data in parsed_data[B]],
#                   'gurobi': [modelo_cda_gurobi(data) for B in parsed_data for data in parsed_data[B]]}}

modelos = {'SCF': {'cplex': {B: [modelo_scf_cplex(data) for data in parsed_data[B]] for B in parsed_data},
                   'gurobi': {B: [modelo_scf_gurobi(data) for data in parsed_data[B]] for B in parsed_data}},
           'CDA': {'cplex': {B: [modelo_cda_cplex(data) for data in parsed_data[B]] for B in parsed_data},
                   'gurobi': {B: [modelo_cda_gurobi(data) for data in parsed_data[B]] for B in parsed_data}}}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-09


In [ ]:
def get_metrics_cpx(model_instance, benchmark, instance):
    start_time = time.time()
    model_instance.solve()
    end_time = time.time()
    return {
        "Modelo": model_instance.name,
        "Benchmark": benchmark,
        "Instancia": instance,
        "Número de Variables": model_instance.number_of_variables,
        "Número de Restricciones": model_instance.number_of_constraints,
        "Valor Función Objetivo": model_instance.objective_value if model_instance.solution is not None else "N/A",
        "Tiempo de Cómputo (s)": end_time - start_time
    }

In [ ]:
def get_metrics_grb(model_instance, benchmark, instance):
    start_time = time.time()
    model_instance.optimize()
    end_time = time.time()
    return {
        "Modelo": model_instance.getAttr("ModelName"),
        "Benchmark": benchmark,
        "Instancia": instance,
        "Número de Variables": len(model_instance.getVars()),
        "Número de Restricciones": len(model_instance.getConstrs()),
        "Valor Función Objetivo": model_instance.objVal if model_instance.status == GRB.OPTIMAL else "N/A",
        "Tiempo de Cómputo (s)": end_time - start_time
    }

In [ ]:
# Ejecución de modelos
for model in modelos:
    for solver in modelos[model]:
        for B in modelos[model][solver]:
            for i, model_instance in enumerate(modelos[model][solver][B]):
                if solver == 'cplex':
                    metrics = get_metrics_cpx(model_instance, B, instances[B][i])
                elif solver == 'gurobi':
                    metrics = get_metrics_grb(model_instance, B, instances[B][i])

                print(metrics)
                file.write(f"{metrics['Modelo']},{metrics['Benchmark']},{metrics['Instancia']},{metrics['Número de Variables']},"
                           f"{metrics['Número de Restricciones']},{metrics['Valor Función Objetivo']},"
                           f"{metrics['Tiempo de Cómputo (s)']}\n")
                file.flush()